In [2]:
import cv2
import numpy as np
import os
from skimage.metrics import structural_similarity as ssim

# Load and preprocess images
def preprocess_image(image_path):
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    if image is None:
        print(f"Error: Unable to load image '{image_path}'")
        return None
    image = cv2.resize(image, (400, 200))  # Resize to standard size
    return image

# Compare two images using SSIM
def compare_images(image1, image2):
    score, diff = ssim(image1, image2, full=True)
    diff = (diff * 255).astype("uint8")
    return score, diff

# Apply threshold and find contours
def detect_fake_pan(diff):
    _, thresh = cv2.threshold(diff, 50, 255, cv2.THRESH_BINARY)
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    return len(contours) > 10  # If too many differences, likely fake

# Load reference PAN card (original format)
reference_image_path = 'ori.jpg'  # Path to the valid PAN card image
if not os.path.exists(reference_image_path):
    print(f"Error: Reference image '{reference_image_path}' not found!")
    exit()

reference_image = preprocess_image(reference_image_path)

# Function to check each PAN card image
def detect_pan_card(test_image_path):
    test_image = preprocess_image(test_image_path)
    if test_image is None:
        return "Error: Could not load image", 0.0

    score, diff = compare_images(reference_image, test_image)
    result = "Valid PAN Card" if score > 0.75 and not detect_fake_pan(diff) else "Fake PAN Card"
    return result, score

# Manually input multiple image paths
def process_selected_images():
    image_paths = input("Enter the paths of the images (comma-separated): ").split(',')

    if not image_paths:
        print("No images provided.")
        return

    for img_path in image_paths:
        img_path = img_path.strip()  # Remove extra spaces
        if not os.path.exists(img_path):
            print(f"Error: File '{img_path}' not found!")
            continue

        result, score = detect_pan_card(img_path)
        print(f"Image: {os.path.basename(img_path)}")
        print(f"SSIM Score: {score:.4f}")
        print(f"Result: {result}")
        print("-" * 40)

# Run the program
process_selected_images()


Enter the paths of the images (comma-separated): 1.png, 2.png, 10.png, 11.png, ori.jpg
Image: 1.png
SSIM Score: 0.2789
Result: Fake PAN Card
----------------------------------------
Image: 2.png
SSIM Score: 0.3463
Result: Fake PAN Card
----------------------------------------
Image: 10.png
SSIM Score: 0.2908
Result: Fake PAN Card
----------------------------------------
Image: 11.png
SSIM Score: 0.2427
Result: Fake PAN Card
----------------------------------------
Image: ori.jpg
SSIM Score: 1.0000
Result: Valid PAN Card
----------------------------------------
